In [2]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from utils.scenario_gen import *
from utils.trees import *
from utils.clustering import *
from utils.optimization import *

In [3]:
datapath = Path().resolve().parent.parent.parent / 'data'
datapath

WindowsPath('D:/Users/Heijden/Documents/PhD work/Current work/DR_IJmuiden/data')

In [4]:
scenario_path = datapath / 'forecast data' / 'Scenarios'
market_datapath = datapath / 'market data'
wl_datapath = datapath / 'waterlevel data'
wb_datapath = datapath / 'waterboard data'
ark_datapath = datapath / 'ark data'

In [5]:
# Start with making input for the optimization problem

In [6]:
distance_metric = 'energy'
scenario_params = {
    'DAM': {
        'n': 1,
        'method': distance_metric,
    },
    'IDM':{
        'n_init': 100,
        'n': 1,
        'method': distance_metric,
    },
    'wl': {
        'n': 1,
        'method': distance_metric,
    },
    'discharge': {
        'n': 1,
        'n_wb': 3,
        'method': distance_metric
    },
    'n_root_steps': 3
}
obs_data = ObservationData(market_datapath, wl_datapath, wb_datapath, ark_datapath)
idm_scenarios = IDMScenarios(
    obs_dataclass=obs_data,
    n_init=scenario_params['IDM']['n_init'],
    n=scenario_params['IDM']['n'],
    method=scenario_params['IDM']['method'],
    cluster=True,
)

scenario_data = ScenarioData(
    scenario_params=scenario_params,
    scenario_path=scenario_path,
    constrain_tree_complexity=True,
    complexity_reduction=0.5,
)

In [20]:
obs_data.wl

,WL
2016-01-01 01:00:00+01:00,0.37
2016-01-01 01:10:00+01:00,0.49
2016-01-01 01:20:00+01:00,0.32
2016-01-01 01:30:00+01:00,0.24
2016-01-01 01:40:00+01:00,0.37
...,...
2021-02-01 21:50:00+01:00,0.72
2021-02-01 22:00:00+01:00,0.66
2021-02-01 22:10:00+01:00,0.59
2021-02-01 22:20:00+01:00,0.55


In [8]:
idx = pd.IndexSlice

In [45]:
wl = scenario_data.wl.copy()
for date in tqdm(wl.index):
    wl.loc[idx[date[0], 0], :] = np.array([obs_data.wl.loc[date[0] + pd.DateOffset(hours=i)].values.flatten()[0]*100  for i in range(1,49)] + [1])

wl.to_pickle(scenario_path / 'WL_obs_nan.pkl')

  0%|          | 0/18260 [00:00<?, ?it/s]

100%|██████████| 18260/18260 [01:02<00:00, 294.45it/s]


In [46]:
wl.interpolate(method='linear', axis=1, limit_direction='both').to_pickle(scenario_path / 'WL_obs.pkl')

In [47]:
wl.interpolate(method='linear', axis=1, limit_direction='both')

,,WL +1,WL +2,WL +3,WL +4,WL +5,WL +6,WL +7,WL +8,WL +9,WL +10,...,WL +40,WL +41,WL +42,WL +43,WL +44,WL +45,WL +46,WL +47,WL +48,weights
date,scenario,,,,,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00+01:00,0,86.0,72.0,54.0,36.0,12.0,4.0,6.0,15.0,33.0,38.0,...,1.0,-46.0,-58.0,-69.0,-79.0,-95.0,-93.0,-69.0,-19.0,1.0
2019-01-01 02:00:00+01:00,0,72.0,54.0,36.0,12.0,4.0,6.0,15.0,33.0,38.0,90.0,...,-46.0,-58.0,-69.0,-79.0,-95.0,-93.0,-69.0,-19.0,51.0,1.0
2019-01-01 03:00:00+01:00,0,54.0,36.0,12.0,4.0,6.0,15.0,33.0,38.0,90.0,142.0,...,-58.0,-69.0,-79.0,-95.0,-93.0,-69.0,-19.0,51.0,77.0,1.0
2019-01-01 04:00:00+01:00,0,36.0,12.0,4.0,6.0,15.0,33.0,38.0,90.0,142.0,137.0,...,-69.0,-79.0,-95.0,-93.0,-69.0,-19.0,51.0,77.0,60.0,1.0
2019-01-01 05:00:00+01:00,0,12.0,4.0,6.0,15.0,33.0,38.0,90.0,142.0,137.0,122.0,...,-79.0,-95.0,-93.0,-69.0,-19.0,51.0,77.0,60.0,34.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-30 16:00:00+01:00,0,72.0,119.0,89.0,59.0,31.0,14.0,5.0,-23.0,-50.0,-58.0,...,86.0,54.0,17.0,-3.0,1.0,-21.0,-58.0,-67.0,-57.0,1.0
2021-01-30 17:00:00+01:00,0,119.0,89.0,59.0,31.0,14.0,5.0,-23.0,-50.0,-58.0,-58.0,...,54.0,17.0,-3.0,1.0,-21.0,-58.0,-67.0,-57.0,-29.0,1.0
2021-01-30 18:00:00+01:00,0,59.0,59.0,31.0,14.0,5.0,-23.0,-50.0,-58.0,-58.0,-59.0,...,17.0,-3.0,1.0,-21.0,-58.0,-67.0,-57.0,-29.0,56.0,1.0


In [28]:
discharge = scenario_data.discharge.copy()
for date in tqdm(discharge.index):
    discharge.loc[idx[date[0], 0], :] = np.array([obs_data.discharge.loc[date[0] + pd.DateOffset(hours=i)].values.flatten()[0]  for i in range(1,49)] + [1])

discharge.to_pickle(scenario_path / 'discharge_obs.pkl')

  0%|          | 0/18260 [00:00<?, ?it/s]

100%|██████████| 18260/18260 [01:02<00:00, 290.57it/s]


In [41]:
discharge.index

MultiIndex([('2019-01-01 01:00:00+01:00', 0),
            ('2019-01-01 02:00:00+01:00', 0),
            ('2019-01-01 03:00:00+01:00', 0),
            ('2019-01-01 04:00:00+01:00', 0),
            ('2019-01-01 05:00:00+01:00', 0),
            ('2019-01-01 06:00:00+01:00', 0),
            ('2019-01-01 07:00:00+01:00', 0),
            ('2019-01-01 08:00:00+01:00', 0),
            ('2019-01-01 09:00:00+01:00', 0),
            ('2019-01-01 10:00:00+01:00', 0),
            ...
            ('2021-01-30 11:00:00+01:00', 0),
            ('2021-01-30 12:00:00+01:00', 0),
            ('2021-01-30 13:00:00+01:00', 0),
            ('2021-01-30 14:00:00+01:00', 0),
            ('2021-01-30 15:00:00+01:00', 0),
            ('2021-01-30 16:00:00+01:00', 0),
            ('2021-01-30 17:00:00+01:00', 0),
            ('2021-01-30 18:00:00+01:00', 0),
            ('2021-01-30 19:00:00+01:00', 0),
            ('2021-01-30 20:00:00+01:00', 0)],
           names=['date', 'scenario'], length=18260)

In [18]:
dam = scenario_data.dam.copy()
for date in tqdm(dam.index):
    dam.loc[idx[date[0], 0], :] = np.array([obs_data.dam.loc[date[0] + pd.DateOffset(hours=i)].values.flatten()[0]  for i in range(48)] + [1])

dam_index0 = dam.index.get_level_values(0)
dam_index1 = dam.index.get_level_values(1)
dam_index0 = dam_index0 - pd.DateOffset(days=1)
dam.index = pd.MultiIndex.from_arrays([dam_index0, dam_index1], names=['date', 'scenario'])
dam.to_pickle(scenario_path / 'DAM_obs.pkl')

100%|██████████| 760/760 [00:02<00:00, 303.04it/s]


In [26]:
scenario_path

WindowsPath('D:/Users/Heijden/Documents/PhD work/Current work/DR_IJmuiden/data/forecast data/Scenarios')

In [15]:
obs_data.dam

,DAM
2018-01-01 00:00:00+01:00,27.20
2018-01-01 01:00:00+01:00,27.30
2018-01-01 02:00:00+01:00,30.10
2018-01-01 03:00:00+01:00,20.87
2018-01-01 04:00:00+01:00,25.56
...,...
2023-01-01 19:00:00+01:00,54.95
2023-01-01 20:00:00+01:00,62.00
2023-01-01 21:00:00+01:00,52.93
2023-01-01 22:00:00+01:00,45.96


In [39]:
dam.dropna().index

MultiIndex([('2019-01-01 00:00:00+01:00', 0),
            ('2019-01-02 00:00:00+01:00', 0),
            ('2019-01-03 00:00:00+01:00', 0),
            ('2019-01-04 00:00:00+01:00', 0),
            ('2019-01-05 00:00:00+01:00', 0),
            ('2019-01-06 00:00:00+01:00', 0),
            ('2019-01-07 00:00:00+01:00', 0),
            ('2019-01-08 00:00:00+01:00', 0),
            ('2019-01-09 00:00:00+01:00', 0),
            ('2019-01-10 00:00:00+01:00', 0),
            ...
            ('2021-01-20 00:00:00+01:00', 0),
            ('2021-01-21 00:00:00+01:00', 0),
            ('2021-01-22 00:00:00+01:00', 0),
            ('2021-01-23 00:00:00+01:00', 0),
            ('2021-01-24 00:00:00+01:00', 0),
            ('2021-01-25 00:00:00+01:00', 0),
            ('2021-01-26 00:00:00+01:00', 0),
            ('2021-01-27 00:00:00+01:00', 0),
            ('2021-01-28 00:00:00+01:00', 0),
            ('2021-01-29 00:00:00+01:00', 0)],
           names=['date', 'scenario'], length=760)

In [32]:
idm

,,DAM NL +1,DAM NL +2,DAM NL +3,DAM NL +4,DAM NL +5,DAM NL +6,DAM NL +7,DAM NL +8,DAM NL +9,DAM NL +10,...,DAM NL +40,DAM NL +41,DAM NL +42,DAM NL +43,DAM NL +44,DAM NL +45,DAM NL +46,DAM NL +47,DAM NL +48,weights
date,scenario,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00+01:00,0,85.51,47.96,48.56,51.09,38.99,48.18,60.40,45.02,54.64,40.48,...,87.49,94.69,95.38,96.26,96.30,91.05,76.85,56.66,47.00,1.0
2019-01-02 00:00:00+01:00,0,43.22,41.52,39.36,39.46,39.31,45.09,53.09,54.80,60.00,54.51,...,67.28,66.90,73.48,88.90,83.43,69.89,64.50,59.00,49.83,1.0
2019-01-03 00:00:00+01:00,0,47.52,45.14,45.25,45.73,46.45,48.56,54.29,67.66,78.22,82.41,...,76.39,86.67,96.28,69.30,52.35,50.20,49.07,48.46,44.28,1.0
2019-01-04 00:00:00+01:00,0,51.22,50.33,50.18,49.11,47.99,48.81,53.62,58.37,57.18,75.36,...,57.61,59.96,69.68,67.60,47.14,49.99,47.18,51.96,51.59,1.0
2019-01-05 00:00:00+01:00,0,60.00,39.61,42.89,45.15,44.62,55.00,45.85,45.57,35.91,47.29,...,50.51,55.46,61.18,62.55,60.74,58.25,45.23,49.32,46.29,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-25 00:00:00+01:00,0,42.82,43.15,43.39,44.26,44.90,48.63,60.04,71.82,95.16,89.34,...,56.81,47.66,42.17,45.11,58.77,63.39,50.36,48.30,46.30,1.0
2021-01-26 00:00:00+01:00,0,44.63,45.40,42.52,44.89,45.85,50.26,NaN,69.24,72.73,71.15,...,120.21,84.43,81.14,76.41,69.39,60.18,53.62,57.15,53.68,1.0
2021-01-27 00:00:00+01:00,0,42.84,42.01,41.41,39.76,39.39,39.25,38.68,45.64,61.94,63.20,...,47.53,52.72,60.18,63.25,59.94,55.20,52.72,51.16,47.34,1.0


In [34]:
idm = dam.copy()
for date in tqdm(idm.index):
    idm.loc[idx[date[0], 0], :] = np.array([obs_data.idm.loc[date[0] + pd.DateOffset(hours=i)].values.flatten()[0]  for i in range(48)] + [1])
idm.to_pickle(scenario_path / 'IDM_obs_nan.pkl')


100%|██████████| 760/760 [00:02<00:00, 296.91it/s]


In [35]:
idm.fillna(1000).to_pickle(scenario_path / 'IDM_obs.pkl')

In [38]:
idm.fillna(1000).dropna()

,,DAM NL +1,DAM NL +2,DAM NL +3,DAM NL +4,DAM NL +5,DAM NL +6,DAM NL +7,DAM NL +8,DAM NL +9,DAM NL +10,...,DAM NL +40,DAM NL +41,DAM NL +42,DAM NL +43,DAM NL +44,DAM NL +45,DAM NL +46,DAM NL +47,DAM NL +48,weights
date,scenario,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00+01:00,0,85.51,47.96,48.56,51.09,38.99,48.18,60.40,45.02,54.64,40.48,...,87.49,94.69,95.38,96.26,96.30,91.05,76.85,56.66,47.00,1.0
2019-01-02 00:00:00+01:00,0,43.22,41.52,39.36,39.46,39.31,45.09,53.09,54.80,60.00,54.51,...,67.28,66.90,73.48,88.90,83.43,69.89,64.50,59.00,49.83,1.0
2019-01-03 00:00:00+01:00,0,47.52,45.14,45.25,45.73,46.45,48.56,54.29,67.66,78.22,82.41,...,76.39,86.67,96.28,69.30,52.35,50.20,49.07,48.46,44.28,1.0
2019-01-04 00:00:00+01:00,0,51.22,50.33,50.18,49.11,47.99,48.81,53.62,58.37,57.18,75.36,...,57.61,59.96,69.68,67.60,47.14,49.99,47.18,51.96,51.59,1.0
2019-01-05 00:00:00+01:00,0,60.00,39.61,42.89,45.15,44.62,55.00,45.85,45.57,35.91,47.29,...,50.51,55.46,61.18,62.55,60.74,58.25,45.23,49.32,46.29,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-25 00:00:00+01:00,0,42.82,43.15,43.39,44.26,44.90,48.63,60.04,71.82,95.16,89.34,...,56.81,47.66,42.17,45.11,58.77,63.39,50.36,48.30,46.30,1.0
2021-01-26 00:00:00+01:00,0,44.63,45.40,42.52,44.89,45.85,50.26,1000.00,69.24,72.73,71.15,...,120.21,84.43,81.14,76.41,69.39,60.18,53.62,57.15,53.68,1.0
2021-01-27 00:00:00+01:00,0,42.84,42.01,41.41,39.76,39.39,39.25,38.68,45.64,61.94,63.20,...,47.53,52.72,60.18,63.25,59.94,55.20,52.72,51.16,47.34,1.0


In [8]:
obs_data.wl

,WL
2016-01-01 01:00:00+01:00,0.37
2016-01-01 01:10:00+01:00,0.49
2016-01-01 01:20:00+01:00,0.32
2016-01-01 01:30:00+01:00,0.24
2016-01-01 01:40:00+01:00,0.37
...,...
2021-02-01 21:50:00+01:00,0.72
2021-02-01 22:00:00+01:00,0.66
2021-02-01 22:10:00+01:00,0.59
2021-02-01 22:20:00+01:00,0.55


In [7]:
scenario_data.wl

,,WL +1,WL +2,WL +3,WL +4,WL +5,WL +6,WL +7,WL +8,WL +9,WL +10,...,WL +40,WL +41,WL +42,WL +43,WL +44,WL +45,WL +46,WL +47,WL +48,weights
date,scenario,,,,,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00+01:00,0,86.36,67.08,52.78,47.73,12.17,-0.89,-13.80,9.15,-4.08,32.96,...,12.99,-9.34,-19.80,-38.19,-43.37,-54.06,-52.43,-10.19,78.35,1.0
2019-01-01 02:00:00+01:00,0,66.14,54.23,30.95,6.01,-21.15,-37.32,-14.73,-10.94,29.02,75.99,...,15.27,14.25,-3.55,-30.31,-32.51,-45.95,-17.50,58.42,128.89,1.0
2019-01-01 03:00:00+01:00,0,50.52,25.82,7.21,-8.98,-32.34,-13.88,0.88,46.63,87.01,104.93,...,-18.90,-27.53,-48.01,-45.12,-13.91,0.40,64.67,105.16,102.36,1.0
2019-01-01 04:00:00+01:00,0,33.68,17.82,1.21,-15.76,-16.68,0.60,35.61,103.05,113.94,84.24,...,-31.75,-43.78,-43.58,-35.02,5.10,40.89,94.06,100.77,108.64,1.0
2019-01-01 05:00:00+01:00,0,7.71,-17.73,-32.75,-23.98,-2.19,39.13,83.81,110.22,86.87,41.88,...,-57.07,-55.22,-34.27,0.15,63.27,114.36,108.64,71.25,52.99,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-06 18:00:00+02:00,0,90.34,62.74,42.23,15.92,6.05,-18.20,-47.93,-63.29,-53.76,-46.76,...,15.69,-5.10,-14.17,-34.48,-62.98,-84.67,-78.51,-50.03,10.13,1.0
2019-04-06 19:00:00+02:00,0,65.63,36.44,19.24,10.07,-16.07,-49.22,-64.83,-61.01,-57.71,-10.12,...,-11.62,-27.48,-34.65,-58.03,-74.82,-71.22,-59.07,-8.13,72.26,1.0
2019-04-06 20:00:00+02:00,0,40.18,24.18,14.60,-13.47,-46.75,-63.99,-57.65,-65.22,-9.31,81.58,...,-17.93,-30.30,-62.94,-75.80,-71.39,-54.81,1.93,81.26,96.78,1.0


In [6]:
# Start with the input DF for the optimization -> a 'scenario df' with the observations
scenario_df = pd.read_pickle(scenario_path / 'WL_energy_1.pkl')
obs_df = pd.read_pickle(wl_datapath / )

SyntaxError: invalid syntax (1813105679.py, line 3)

In [12]:
example_df

,,WL +1,WL +2,WL +3,WL +4,WL +5,WL +6,WL +7,WL +8,WL +9,WL +10,...,WL +40,WL +41,WL +42,WL +43,WL +44,WL +45,WL +46,WL +47,WL +48,weights
date,scenario,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00+00:00,0,86.36,67.08,52.78,47.73,12.17,-0.89,-13.80,9.15,-4.08,32.96,...,12.99,-9.34,-19.80,-38.19,-43.37,-54.06,-52.43,-10.19,78.35,1.0
2019-01-01 01:00:00+00:00,0,66.14,54.23,30.95,6.01,-21.15,-37.32,-14.73,-10.94,29.02,75.99,...,15.27,14.25,-3.55,-30.31,-32.51,-45.95,-17.50,58.42,128.89,1.0
2019-01-01 02:00:00+00:00,0,50.52,25.82,7.21,-8.98,-32.34,-13.88,0.88,46.63,87.01,104.93,...,-18.90,-27.53,-48.01,-45.12,-13.91,0.40,64.67,105.16,102.36,1.0
2019-01-01 03:00:00+00:00,0,33.68,17.82,1.21,-15.76,-16.68,0.60,35.61,103.05,113.94,84.24,...,-31.75,-43.78,-43.58,-35.02,5.10,40.89,94.06,100.77,108.64,1.0
2019-01-01 04:00:00+00:00,0,7.71,-17.73,-32.75,-23.98,-2.19,39.13,83.81,110.22,86.87,41.88,...,-57.07,-55.22,-34.27,0.15,63.27,114.36,108.64,71.25,52.99,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-30 17:00:00+00:00,0,71.65,41.71,12.31,2.48,-10.72,-44.66,-54.45,-63.37,-59.94,-38.76,...,-36.00,-46.00,-66.48,-84.88,-108.33,-93.63,-86.70,-47.15,32.12,1.0
2021-01-30 18:00:00+00:00,0,41.56,20.34,10.02,-10.28,-47.44,-67.15,-56.88,-52.10,-29.94,62.87,...,-61.81,-56.37,-72.31,-105.81,-105.76,-83.18,-27.36,54.30,100.79,1.0
2021-01-30 19:00:00+00:00,0,14.91,0.54,-25.13,-53.85,-76.20,-71.03,-62.16,-47.44,42.42,99.11,...,-60.69,-87.39,-113.88,-112.54,-106.33,-50.55,49.65,88.46,61.49,1.0
